### データセットのダウンロード
https://github.com/oreilly-japan/ml-security-jp/blob/master/ch03/archive.zip  
を取得して同じディレクトリに格納し、解凍を行う。  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
AndroidDataset = pd.read_csv('train.csv', sep=';')

In [ ]:
AndroidDataset.type.value_counts()

In [ ]:
print(AndroidDataset.columns)

In [ ]:
pd.Series.sort_values(AndroidDataset[AndroidDataset.type == 1].sum(axis=0), ascending=False)[1:11] # 0は含まない(typeが全て1でこれがTopなのは当然)

In [ ]:
top10 = [
    'android.permission.INTERNET',
    'android.permission.READ_PHONE_STATE',
    'android.permission.ACCESS_NETWORK_STATE',
    'android.permission.WRITE_EXTERNAL_STORAGE',
    'android.permission.ACCESS_WIFI_STATE',
    'android.permission.READ_SMS',
    'android.permission.WRITE_SMS',
    'android.permission.RECEIVE_BOOT_COMPLETED',
    'android.permission.ACCESS_COARSE_LOCATION',
    'android.permission.CHANGE_WIFI_STATE',
]

AndroidDataset.loc[AndroidDataset.type == 0, top10].sum()

In [ ]:
fig, axs = plt.subplots(nrows=2, sharex=True)

AndroidDataset.loc[AndroidDataset.type == 0, top10].sum().plot(kind='bar', ax=axs[0])
AndroidDataset.loc[AndroidDataset.type == 1, top10].sum().plot(kind='bar', ax=axs[1], color='red')